In [1]:
import pandas as pd
import numpy as np
import math
import pyomo.environ as pyo 
import pyomo.opt as opt

In [2]:

ir = pd.read_excel("/Users/Jarusch/Documents/Hertie/Hertie School/OneDrive - Hertie School/Jarusch/CRIME/Modelling/Data/DATA.XLSM", index_col = False, sheet_name =8)

In [3]:
ir

,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,...,2091,2092,2093,2094,2095,2096,2097,2098,2099,2100
0,1.05,1.1025,1.157625,1.215506,1.276282,1.340096,1.4071,1.477455,1.551328,1.628895,...,33.545134,35.222391,36.98351,38.832686,40.77432,42.813036,44.953688,47.201372,49.561441,52.039513


In [4]:
raw = pd.read_excel("/Users/Jarusch/Documents/Hertie/Hertie School/OneDrive - Hertie School/Jarusch/CRIME/Modelling/Data/DATA.XLSM", index_col =0 , sheet_name =[1, 2, 3, 4, 5, 6, 7])

In [5]:
A = raw[1]
f = raw[2]
F = raw[3]
p = raw[4]
B = raw[5]
c = raw[6]
Q = raw[7]

In [6]:
p.head()

,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,...,2091,2092,2093,2094,2095,2096,2097,2098,2099,2100
Provide ore ($ / t),0.0,0.000000,0.000000,0.0,0.000000,0.000000,0,0.000000,0.000000,0.0,...,0.000,0.000,0.000,0.000,0.00,0.000,0.000,0.000,0.000,0.0
Provide coal ($ / t),105.0,105.333333,105.666667,106.0,106.333333,106.666667,107,107.333333,107.666667,108.0,...,143.700,144.400,145.100,145.800,146.50,147.200,147.900,148.600,149.300,150.0
Provide H2 ($ / kg),3.6,3.500000,3.400000,3.3,3.200000,3.100000,3,2.900000,2.800000,2.7,...,1.354,1.348,1.342,1.336,1.33,1.324,1.318,1.312,1.306,1.3
Provide electricity ($/ kWh),0.0,0.000000,0.000000,0.0,0.000000,0.000000,0,0.000000,0.000000,0.0,...,0.000,0.000,0.000,0.000,0.00,0.000,0.000,0.000,0.000,0.0
Buy CO2 price ($ / t),0.0,0.000000,0.000000,0.0,0.000000,0.000000,0,0.000000,0.000000,0.0,...,0.000,0.000,0.000,0.000,0.00,0.000,0.000,0.000,0.000,0.0


In [7]:
raw_energy = pd.read_excel("/Users/Jarusch/Documents/Hertie/Hertie School/OneDrive - Hertie School/Jarusch/CRIME/Modelling/Data/energy_costs.xlsx")
raw_energy = raw_energy.iloc[0, 8:-1].to_frame().reindex(index = list(range(2020, 2101)))

In [8]:
ts = pd.to_numeric(raw_energy.iloc[:, 0])

ts = ts.interpolate()

energy_p = ts * 1/277.778 * 1/math.pow(ir.iloc[0,0], 5)

In [9]:
raw_carbon = pd.read_excel("/Users/Jarusch/Documents/Hertie/Hertie School/OneDrive - Hertie School/Jarusch/CRIME/Modelling/Data/carbon_prices.xlsx", index_col = "Scenario")
raw_carbon = raw_carbon.iloc[0:5, 6:-1].transpose().reindex(index = list(range(2020, 2101)))

In [10]:
carbon_p = raw_carbon.interpolate() * 1/ math.pow(ir.iloc[0,0], 5)

In [11]:
p.loc["Provide electricity ($/ kWh)", :]  = energy_p

In [12]:
p_ssp1 = p
p_ssp1.loc["Buy CO2 price ($ / t)", :] = carbon_p.loc[:, "SSP1-19"]

p_ssp2 = p
p_ssp2.loc["Buy CO2 price ($ / t)", :] = carbon_p.loc[:, "SSP2-19"]

p_ssp3 = p
p_ssp3.loc["Buy CO2 price ($ / t)", :] = carbon_p.loc[:, "SSP3-34"]

p_ssp4 = p
p_ssp4.loc["Buy CO2 price ($ / t)", :] = carbon_p.loc[:, "SSP4-26"]

p_ssp5 = p
p_ssp5.loc["Buy CO2 price ($ / t)", :] = carbon_p.loc[:, "SSP5-19"]

In [13]:
p_ssp1

,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,...,2091,2092,2093,2094,2095,2096,2097,2098,2099,2100
Provide ore ($ / t),0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Provide coal ($ / t),105.000000,105.333333,105.666667,106.000000,106.333333,106.666667,107.000000,107.333333,107.666667,108.000000,...,143.700000,144.400000,145.100000,145.800000,146.500000,147.200000,147.900000,148.600000,149.300000,150.000000
Provide H2 ($ / kg),3.600000,3.500000,3.400000,3.300000,3.200000,3.100000,3.000000,2.900000,2.800000,2.700000,...,1.354000,1.348000,1.342000,1.336000,1.330000,1.324000,1.318000,1.312000,1.306000,1.300000
Provide electricity ($/ kWh),0.058738,0.065451,0.072165,0.078879,0.085593,0.092307,0.094992,0.097678,0.100363,0.103049,...,0.075264,0.075408,0.075551,0.075695,0.075839,0.075983,0.076126,0.076270,0.076414,0.076558
Buy CO2 price ($ / t),5.745597,12.717962,19.690326,26.662690,33.635055,40.607419,47.579783,54.552147,61.524512,68.496876,...,3885.741317,4103.796649,4321.851982,4539.907314,4757.962646,4976.017978,5194.073310,5412.128642,5630.183974,5848.239307


In [14]:
y = f## * f

In [18]:
Processes = ["Conventional process", "Directreduction", "Ore dummy", "Coal dummy 1", "H2 dummy", "electrcity dummy"]

In [19]:
mdl = pyo.ConcreteModel()

mdl.P = pyo.Set(initialize=Processes)

mdl.s_p = mdl.Var(mdl.P, domain=pyo.NonNegativeIntegers)


In [30]:
def fulfill_demand (mdl, p):
    return A * mdl.s_p >= y

mdl.demand_restriction = pyo.Constraint(mdl.P, rule=fulfill_demand)


    (type=<class 'pyomo.core.base.constraint.IndexedConstraint'>) on block
    unknown with a new Component (type=<class
    'pyomo.core.base.constraint.IndexedConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
ERROR: Rule failed when generating expression for constraint
    demand_restriction with index Conventional process: TypeError: <class
    'pyomo.core.base.block._generic_component_decorator'>
ERROR: Constructing component 'demand_restriction' from data=None failed:
    TypeError: <class 'pyomo.core.base.block._generic_component_decorator'>


TypeError: <class 'pyomo.core.base.block._generic_component_decorator'>

,Provide ore ($ / t),Provide coal ($ / t),Provide H2 ($ / kg),Provide electricity ($/ kWh),Buy CO2 price ($ / t)
2020,0.0,105.0,3.6,0.058738,5.745597


In [45]:
def min_costs(mdl): 
    return mdl.P * F * p.iloc[:,0].to_frame().transpose()

mdl.Obj = pyo.Objective(rule=min_costs, sense=pyo.minimize)



In [49]:
solvername='glpk'

solverpath_folder= "/Users/Jarusch/opt/anaconda3/lib/python3.8/site-packages/pyomo/solvers/plugins/solvers"

#solverpath_exe='/Users/Jarusch/opt/anaconda3/lib/python3.8/site-packages/pyomo/solvers/plugins/solvers' #does not need to be directly on c drive

sys.path.append(solverpath_folder)

solver=SolverFactory(solvername)

NameError: name 'sys' is not defined

In [50]:
optimizer = opt.SolverFactory('glpk') 
solved_model = optimizer.solve(mdl)



    solver 'glpk'


ApplicationError: No executable found for solver 'glpk'

In [48]:
print("Optimal value: %i" % (round(mdl.Obj.expr(),2))) #threshold to avoid numerical issues
EPS = 1.e-6

if mdl.getVal(x_i_j[i,j]) > EPS:
    print("run the processes according to: %s" % (pyomo.value(model.s)))
    
    

TypeError: 'SetProduct_FiniteSet' object is not callable